In [11]:
import numpy as np 
import pandas as pd 
data_files = ['ap_2010.csv','class_size.csv','graduation.csv','demographics.csv','hs_directory.csv','math_test_results.csv','sat_results.csv']
data = {}
for i in data_files:
    df= pd.read_csv("Datasets/{0}".format(i))
    print i
    data[i.replace(".csv","")] = df

ap_2010.csv
class_size.csv
graduation.csv
demographics.csv
hs_directory.csv
math_test_results.csv
sat_results.csv


In [12]:
for k,v in data.items():
    print k+"\n"
    print v.head()

math_test_results

      DBN Grade  Year      Category  Number Tested Mean Scale Score Level 1 #  \
0  01M015     3  2006  All Students             39              667         2   
1  01M015     3  2007  All Students             31              672         2   
2  01M015     3  2008  All Students             37              668         0   
3  01M015     3  2009  All Students             33              668         0   
4  01M015     3  2010  All Students             26              677         6   

  Level 1 % Level 2 # Level 2 % Level 3 # Level 3 % Level 4 # Level 4 %  \
0      5.1%        11     28.2%        20     51.3%         6     15.4%   
1      6.5%         3      9.7%        22       71%         4     12.9%   
2        0%         6     16.2%        29     78.4%         2      5.4%   
3        0%         4     12.1%        28     84.8%         1        3%   
4     23.1%        12     46.2%         6     23.1%         2      7.7%   

  Level 3+4 # Level 3+4 %  
0          26  

In [13]:
for k,v in data.items():
    try:
        print k+"\n"
        print v["DBN"].head(2)
    except KeyError:
        print "Column not found"
print data['class_size'].head()

math_test_results

0    01M015
1    01M015
Name: DBN, dtype: object
sat_results

0    01M292
1    01M448
Name: DBN, dtype: object
demographics

0    01M015
1    01M015
Name: DBN, dtype: object
graduation

0    01M292
1    01M292
Name: DBN, dtype: object
hs_directory

Column not found
ap_2010

0    01M448
1    01M450
Name: DBN, dtype: object
class_size

Column not found
   CSD BOROUGH SCHOOL CODE                SCHOOL NAME GRADE  PROGRAM TYPE  \
0    1       M        M015  P.S. 015 Roberto Clemente     0K       GEN ED   
1    1       M        M015  P.S. 015 Roberto Clemente     0K          CTT   
2    1       M        M015  P.S. 015 Roberto Clemente     01       GEN ED   
3    1       M        M015  P.S. 015 Roberto Clemente     01          CTT   
4    1       M        M015  P.S. 015 Roberto Clemente     02       GEN ED   

  CORE SUBJECT (MS CORE and 9-12 ONLY) CORE COURSE (MS CORE and 9-12 ONLY)  \
0                                    -                                   -   
1        

In [14]:
data['class_size']['DBN'] = data['class_size'].apply(lambda x: "{0:02d}{1}".format(x["CSD"], x["SCHOOL CODE"]), axis=1)
print data['class_size']['DBN'].head()

0    01M015
1    01M015
2    01M015
3    01M015
4    01M015
Name: DBN, dtype: object


#### Adding Survey data

In [15]:
survey1 = pd.read_csv("Datasets/survey/masterfile11_d75_final.txt", delimiter='\t', encoding='windows-1252')
survey2 = pd.read_csv("Datasets/survey/masterfile11_gened_final.txt", delimiter='\t', encoding='windows-1252')
survey1["d75"] = True 
survey2["d75"] = False 
survey = pd.concat([survey1,survey2],axis=0)


In [16]:
survey["DBN"] = survey["dbn"]
# Using only informative fields from survey dataset
survey_fields = ["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_10", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11",]

survey = survey.loc[:, survey_fields]
data["survey"] = survey

#### Condensing datasets

In [17]:
class_size = data['class_size']
class_size = class_size[class_size['PROGRAM TYPE']=="GEN ED"]
class_size = class_size.groupby("DBN").agg(np.mean)
class_size.reset_index(inplace=True)
data["class_size"] = class_size

In [18]:
demographics = data['demographics']
demographics = demographics[demographics["schoolyear"]==20112012]
data['demographics'] = demographics

In [19]:
math_results = data["math_test_results"]
math_results = math_results[math_results["Year"]==2011]
math_results = math_results[math_results["Grade"]==8]
data["math_test_results"] = math_results

In [20]:
data['graduation'] = data['graduation'][data['graduation']['Cohort']=="2006"]
data['graduation'] = data['graduation'][data['graduation']['Demographic']=="Total Cohort"]

In [10]:
data['graduation'].head()

,Demographic,DBN,School Name,Cohort,Total Cohort,Total Grads - n,Total Grads - % of cohort,Total Regents - n,Total Regents - % of cohort,Total Regents - % of grads,...,Regents w/o Advanced - n,Regents w/o Advanced - % of cohort,Regents w/o Advanced - % of grads,Local - n,Local - % of cohort,Local - % of grads,Still Enrolled - n,Still Enrolled - % of cohort,Dropped Out - n,Dropped Out - % of cohort
3,Total Cohort,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL,2006,78,43,55.1%,36,46.2%,83.7%,...,36,46.2%,83.7%,7,9%,16.3%,16,20.5%,11,14.1%
10,Total Cohort,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,2006,124,53,42.7%,42,33.9%,79.2%,...,34,27.4%,64.2%,11,8.9%,20.8%,46,37.1%,20,16.100000000000001%
17,Total Cohort,01M450,EAST SIDE COMMUNITY SCHOOL,2006,90,70,77.8%,67,74.400000000000006%,95.7%,...,67,74.400000000000006%,95.7%,3,3.3%,4.3%,15,16.7%,5,5.6%
24,Total Cohort,01M509,MARTA VALLE HIGH SCHOOL,2006,84,47,56%,40,47.6%,85.1%,...,23,27.4%,48.9%,7,8.300000000000001%,14.9%,25,29.8%,5,6%
31,Total Cohort,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHO,2006,193,105,54.4%,91,47.2%,86.7%,...,22,11.4%,21%,14,7.3%,13.3%,53,27.5%,35,18.100000000000001%
